# Translate text with quality checks

**Note:**
To download this example as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/translation_with_quality_check.ipynb).

In this example, we will use Guardrails during the translation of a statement from another language to English. We will check whether the translated statement is likely of high quality.

## Objective

We want to translate a statement from different languages to English and ensure that the translated statement accurately reflects the original content.

### Setup

- Install the `unbabel-comet` from source:
  `pip install git+https://github.com/Unbabel/COMET`
- Please accept the model license from:
  https://huggingface.co/Unbabel/wmt22-cometkiwi-da
- Login into Huggingface Hub using:
  huggingface-cli login --token $HUGGINGFACE_TOKEN


In [1]:
! pip install git+https://github.com/Unbabel/COMET -q

In [2]:
import openai
from pydantic import BaseModel, Field
import guardrails as gd
from rich import print
from guardrails.validators import IsHighQualityTranslation

openai.api_key = "YOUR_API_KEY"

/Users/karanacharya/.pyenv/versions/3.11.6/envs/guard-venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 1: Create the RAIL Spec / Pydantic model

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](../concepts/output.md). We will also show the same RAIL spec in a code-first format using a Pydantic model.

In this RAIL spec, we:

1. Create an `output` schema that returns a single key-value pair. The key should be 'translated_statement', and the value should be the English translation of the given statement. The translated statement should not have any profanity.


Our RAIL spec as an XML string:


In [12]:
# rail_str = """
# <rail version="0.1">

# <output>
#     <string
#         name="translated_statement"
#         description="Translate the given statement into the English language"
#         format="is-high-quality-translation"
#         on-fail-is-high-quality-translation="fix"
#     />
# </output>


# <prompt>
# Translate the given statement into the English language:

# ${statement_to_be_translated}

# ${gr.complete_json_suffix}
# </prompt>
# </rail>
# """

# Or a Pydantic model
prompt = """
Translate the given statement into English:

${statement_to_be_translated}

${gr.complete_json_suffix_v2}
"""


class HighQualityTranslation(BaseModel):
    translated_statement: str = Field(
        description="Validate the translation quality of the given statement",
        validators=[IsHighQualityTranslation(on_fail="fix")],
    )

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 52038.51it/s]



Loading the model from checkpoint...


Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
Encoder model frozen.
/Users/karanacharya/.pyenv/versions/3.11.6/envs/guard-venv/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.


From our RAIL string:


In [4]:
# guard = gd.Guard.from_rail_string(rail_str)

# Or a Pydantic model
guard = gd.Guard.from_pydantic(output_class=HighQualityTranslation, prompt=prompt)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 83220.32it/s]


Loading the model from checkpoint...



Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
Encoder model frozen.


We see the prompt that will be sent to the LLM:


In [5]:
print(guard.base_prompt)

Translate the given statement into English:

${statement_to_be_translated}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="translated_statement" description="Validate the translation quality of the given statement" 
format="is-high-quality-translation"/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

Here, `statement_to_be_translated` is the the statement and will be provided by the user at runtime.


## Step 3: Wrap the LLM API call with `Guard`


First, let's try translating a statement that is relatively easy to translate.


In [6]:
statement = "Ich habe keine Ahnung, was ich hier schreiben soll."

raw_llm_response, validated_response, *rest = guard(
    openai.Completion.create,
    prompt_params={"statement_to_be_translated": statement},
    metadata={"translation_source": statement},
    engine="text-davinci-003",
    max_tokens=1024,
    temperature=0,
)

print(f"Raw LLM Output: {raw_llm_response}")
print(f"Validated Output: {validated_response}")

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karanacharya/.pyenv/versions/3.11.6/envs/guard-venv/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/karanacharya/.pyenv/versions/3.11.6/envs/guard-venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  9.48it/s]


Translation quality: 0.8714022636413574


Raw LLM Output: {"translated_statement": "I have no idea what I should write here."}

Validated Output: {'translated_statement': 'I have no idea what I should write here.'}

We can look at the logs to see the quality check results:


In [7]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │ Translate the given statement into English:                                                             │ │
    │ │                                                                                                         │ │
    │ │ Ich habe keine Ahnung, was ich hier schreiben soll.                                                     │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="translated_statement" description="Validate the translation quality of the given      │ │
    │ │ statement" format="is-high-quality-translation"/>                                                       │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise.                                 │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Json Output:                                                                                            │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ ╰────────────────────────────────────────────

The `guard` wrapper returns the raw LLM response, which is the translated statement and also the validated output. In this case, the translated statement was of a good quality (above the threshold of 0.5), so the validated output is the same as the raw LLM response.

#### Now, let's test with a really low quality translation, and see how Guardrails handles it.


In [8]:
# Test with validation method 'full'
guard = gd.Guard.from_string(
    validators=[IsHighQualityTranslation(on_fail="fix")],
    description="Validate the translation quality of the given statement",
)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 106454.42it/s]


Loading the model from checkpoint...



Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
Encoder model frozen.
/Users/karanacharya/.pyenv/versions/3.11.6/envs/guard-venv/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 87746.95it/s]



Loading the model from checkpoint...


Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.1.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../.cache/huggingface/hub/models--Unbabel--wmt22-cometkiwi-da/snapshots/b3a8aea5a5fc22db68a554b92b3d96eb6ea75cc9/checkpoints/model.ckpt`
Encoder model frozen.


In [9]:
# Parse the code snippet
statement = "अरे भाऊ, आज रात्री जोरदार पार्टी मारूया, जमून टाकूया आणि धमाल करूया!"

## Ideal translation from Marathi -> English:
#  "Hey bro, let's have a great party tonight and have fun!"

output = guard.parse(
    llm_output="It's such a beautiful day, I'm going to the beach.",  ## here, providing a really bad translation
    metadata={"translation_source": statement},
)

# Print the output
print(output)
print(f"Raw LLM Output: {output.raw_llm_output}")
print(f"Validated Output: {output.validated_output}")

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karanacharya/.pyenv/versions/3.11.6/envs/guard-venv/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:187: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/karanacharya/.pyenv/versions/3.11.6/envs/guard-venv/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'predict_dataloader' to speed up the dataloader worker initialization.
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  7.92it/s]


Translation quality: 0.4143546521663666


ValidationOutcome(
    raw_llm_output="It's such a beautiful day, I'm going to the beach.",
    validated_output='',
    reask=None,
    validation_passed=True,
    error=None
)

Raw LLM Output: It's such a beautiful day, I'm going to the beach.

Validated Output:

In [19]:
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │ No prompt                                                                                               │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ No message history.                                                                                     │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ It's such a beautiful day, I'm going to the beach.                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ ''                                                                                                      │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

As you can see, the translation quality is really bad, and the `IsHighQualityTranslation` check failed as the translation quality was below the threshold. The validated response is an empty string.

## In this way, you can use Guardrails to ensure that the output of your LLM is of high quality.
